In [70]:
!pip install openpyxl pandas

In [71]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows

In [ ]:
# Chemins des fichiers
fichier_temoin = "chemin/du/fichier/temoin"
fichier_llm = "chemin/du/fichier/a/corriger"
sortie_raw = "choisir_nom_a_donner_au_fichier_avec_erreurs_annotees"
sortie_clean = "choisir_nom_a_donner_au_fichier_avec_erreurs_corrigees"

In [73]:
# Lecture des CSV
df_temoin = pd.read_csv(fichier_temoin, sep=';')
df_llm = pd.read_csv(fichier_llm)

In [74]:
# Vérification des dimensions
print(f"Témoin: {df_temoin.shape[0]} lignes, {df_temoin.shape[1]} colonnes")
print(f"LLM: {df_llm.shape[0]} lignes, {df_llm.shape[1]} colonnes")

Témoin: 11 lignes, 31 colonnes
LLM: 11 lignes, 31 colonnes


In [75]:
# Identifier les colonnes communes et manquantes
colonnes_communes = [col for col in df_temoin.columns if col in df_llm.columns]
colonnes_manquantes = [col for col in df_temoin.columns if col not in df_llm.columns]

print(f"\nColonnes communes: {len(colonnes_communes)}")
print(f"Colonnes manquantes dans LLM: {len(colonnes_manquantes)}")
if colonnes_manquantes:
    print(f"Liste: {', '.join(colonnes_manquantes)}")


Colonnes communes: 31
Colonnes manquantes dans LLM: 0


In [76]:
# Réorganiser les colonnes du LLM pour matcher le témoin
df_llm = df_llm[colonnes_communes]

In [77]:
def normaliser_valeur(val):
    if pd.isna(val):
        return ""
    val_str = str(val).strip().lower()
    if val_str in ['nr', 'non', 'n/a', 'nan']:
        return "nr"
    # Supprimer les espaces multiples
    val_str = ' '.join(val_str.split())
    # Normaliser les apostrophes (par code Unicode)
    val_str = val_str.replace(chr(8217), "'").replace(chr(8216), "'").replace('`', "'")
    return val_str

In [78]:
# Fonction pour créer un fichier Excel avec mise en forme
def creer_excel_avec_differences(df_source, df_reference, nom_fichier, mode_correction=False, colonnes_manquantes=[]):
    wb = Workbook()
    ws = wb.active
    
    # Bordures pour toutes les cellules
    thin_border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    
    # Utiliser toutes les colonnes du témoin pour le fichier clean
    if mode_correction:
        colonnes_a_utiliser = list(df_reference.columns)
    else:
        colonnes_a_utiliser = list(df_source.columns)
    
    # Écrire les en-têtes
    for col_idx, col_name in enumerate(colonnes_a_utiliser, 1):
        cell = ws.cell(row=1, column=col_idx, value=col_name)
        cell.border = thin_border
        cell.alignment = Alignment(wrap_text=True, vertical='top')
    
    # Écrire les données avec mise en forme
    for row_idx in range(min(len(df_source), len(df_reference))):
        for col_idx, col_name in enumerate(colonnes_a_utiliser, 1):
            cell = ws.cell(row=row_idx + 2, column=col_idx)
            cell.border = thin_border
            cell.alignment = Alignment(wrap_text=True, vertical='top')
            
            # Vérifier si la colonne existe dans les deux dataframes
            col_dans_source = col_name in df_source.columns
            col_dans_reference = col_name in df_reference.columns
            
            if col_dans_source and col_dans_reference:
                val_source = df_source.loc[row_idx, col_name]
                val_reference = df_reference.loc[row_idx, col_name]
                
                # Normaliser pour comparaison
                val_source_norm = normaliser_valeur(val_source)
                val_reference_norm = normaliser_valeur(val_reference)
                
                # Si les valeurs sont identiques (ou équivalentes)
                if val_source_norm == val_reference_norm:
                    cell.value = val_source
                else:
                    # Valeurs différentes
                    if mode_correction:
                        # Mode clean : afficher la valeur corrigée en vert
                        cell.value = val_reference
                        cell.font = Font(bold=True, color="006100")
                    else:
                        # Mode raw : afficher la valeur du LLM en rouge
                        cell.value = val_source
                        cell.font = Font(bold=True, color="9C0006")
            
            elif col_dans_reference and not col_dans_source:
                # Colonne manquante dans le LLM
                if mode_correction:
                    # Dans le fichier clean, ajouter les valeurs du témoin
                    cell.value = df_reference.loc[row_idx, col_name]
                    cell.font = Font(bold=True, color="006100")
                else:
                    # Dans le fichier raw, laisser vide
                    cell.value = ""
    
    # Ajuster la largeur des colonnes
    for column in ws.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 50)
        ws.column_dimensions[column_letter].width = adjusted_width
    
    # Ajouter le message pour les colonnes manquantes (fichier raw uniquement)
    if not mode_correction and colonnes_manquantes:
        message_row = len(df_source) + 3
        message_cell = ws.cell(row=message_row, column=1)
        message_cell.value = f"Il manque {len(colonnes_manquantes)} colonne(s) : {', '.join(colonnes_manquantes)}"
        message_cell.font = Font(bold=True, color="9C0006")
    
    wb.save(nom_fichier)
    print(f"Fichier créé: {nom_fichier}")


In [79]:
# Appel de la fonction
creer_excel_avec_differences(df_llm, df_temoin, sortie_raw, mode_correction=False, colonnes_manquantes=colonnes_manquantes)
creer_excel_avec_differences(df_llm, df_temoin, sortie_clean, mode_correction=True, colonnes_manquantes=colonnes_manquantes)

Fichier créé: resultats_batch0_fitz_Ministral_3B_raw.xlsx
Fichier créé: resultats_batch0_fitz_Ministral_3B_clean.xlsx


In [80]:
creer_excel_avec_differences(df_llm, df_temoin, sortie_raw, mode_correction=False, colonnes_manquantes=colonnes_manquantes)
creer_excel_avec_differences(df_llm, df_temoin, sortie_clean, mode_correction=True, colonnes_manquantes=colonnes_manquantes)

Fichier créé: resultats_batch0_fitz_Ministral_3B_raw.xlsx
Fichier créé: resultats_batch0_fitz_Ministral_3B_clean.xlsx


In [81]:
# Statistiques des différences
differences = 0
total_cellules = 0

for col_name in colonnes_communes:
    for row_idx in range(len(df_llm)):
        total_cellules += 1
        val_llm = normaliser_valeur(df_llm.loc[row_idx, col_name])
        val_temoin = normaliser_valeur(df_temoin.loc[row_idx, col_name])
        if val_llm != val_temoin:
            differences += 1

if total_cellules > 0:
    print(f"\nDifférences trouvées: {differences}/{total_cellules} cellules ({differences/total_cellules*100:.2f}%)")
    print(f"Colonnes manquantes: {len(colonnes_manquantes)}")
else:
    print("\nAucune cellule à comparer.")


Différences trouvées: 118/341 cellules (34.60%)
Colonnes manquantes: 0


In [82]:
# Statistiques des différences (si jamais un des deux fichiers a plus de lignes que l'autre)
differences = 0
total_cellules = 0

for col_name in colonnes_communes:
    for row_idx in range(min(len(df_llm), len(df_temoin))):
        total_cellules += 1
        val_llm = normaliser_valeur(df_llm.loc[row_idx, col_name])
        val_temoin = normaliser_valeur(df_temoin.loc[row_idx, col_name])
        if val_llm != val_temoin:
            differences += 1

if total_cellules > 0:
    print(f"\nDifférences trouvées: {differences}/{total_cellules} cellules ({differences/total_cellules*100:.2f}%)")
    print(f"Colonnes manquantes: {len(colonnes_manquantes)}")
else:
    print("\nAucune cellule à comparer.")


Différences trouvées: 118/341 cellules (34.60%)
Colonnes manquantes: 0
